In [23]:
from data_rnn import load_ndfa, load_brackets
from data_prep import pad_and_convert
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [24]:
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=150_000)
x_train_brackets, (i2w_brackets, w2i_brackets) = load_brackets(n=150_000)

In [64]:
print(''.join([i2w_ndfa[i] for i in  x_train_ndfa[50]]))

ss


In [26]:
print(''.join([i2w_brackets[i] for i in  x_train_brackets[10_000]]) )

()


In [27]:
len(x_train_ndfa[-1])

158

In [28]:
len(i2w_ndfa[-1])

1

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, h, num_char, n_layers=1):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=h, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(h, num_char)
    
    def forward(self, input_seq, h):
        

        embedded = self.embedding(input_seq)
        lstm_out, hidden = self.lstm(embedded, h)
        lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        return output, hidden


In [31]:
def pad_and_convert2(batch, w2i):
    # Get indices for '.start' and '.end' tokens from the w2i dictionary
    start_token = w2i['.start']
    end_token = w2i['.end']
    
    # Prepend '.start' and append '.end' to each sequence in the batch
    batch = [[start_token] + x + [end_token] for x in batch]
    
    # Calculate the maximum length after adding start and end tokens
    max_len = max([len(x) for x in batch])
    
    # Pad the sequences with the '.pad' token
    batch = [x + [w2i['.pad']] * (max_len - len(x)) for x in batch]
    
    # Convert the batch to a PyTorch tensor
    batch = torch.tensor(batch, dtype=torch.long)
    
    return batch

In [32]:
len(x_train_ndfa[-1])

158

In [33]:
len(x_train_ndfa[0])

2

In [34]:
x_train_ndfa_padded = pad_and_convert(x_train_ndfa)
x_train_brackets_padded = pad_and_convert(x_train_brackets)

In [48]:
len(x_train_brackets_padded[-1])

1022

Nieuwe pad and convert, sequence zou toch niet van 158 naar 1022 moeten gaan? Ze zijn op volgorde van klein naar groot al neergezet

Langste sequence was 158 + .start +.end is 160. Alle sequences zijn nu van lengte 160

In [35]:
x_train_ndfa_padded2 = pad_and_convert2(x_train_ndfa, w2i_ndfa)
x_train_brackets_padded2 = pad_and_convert2(x_train_brackets, w2i_brackets)

In [36]:
len(x_train_ndfa_padded2[-1])

160

In [55]:
len(x_train_ndfa_padded2[0])

160

In [37]:
x_train_ndfa_padded[:10]

tensor([[5, 5, 0,  ..., 0, 0, 0],
        [5, 5, 0,  ..., 0, 0, 0],
        [5, 5, 0,  ..., 0, 0, 0],
        ...,
        [5, 5, 0,  ..., 0, 0, 0],
        [5, 5, 0,  ..., 0, 0, 0],
        [5, 5, 0,  ..., 0, 0, 0]])

In [39]:
max_len = max([len(x) for x in x_train_ndfa])
max_len

158

In [40]:
x_train = x_train_ndfa_padded[:, :-1]
y_train = x_train_ndfa_padded[:, -1]

In [41]:
print(type(x_train_ndfa))

<class 'list'>


In [42]:
model = LSTM(vocab_size=len(set(w2i_ndfa)), emb_size=300, h=300, num_char=len(set(w2i_ndfa)), n_layers=1)

In [43]:
vocab_size = len(set(w2i_ndfa))
emb_size = 300
h = 300
num_char = vocab_size
n_layers = 1

In [44]:
num_epochs = 1
learning_rate = 0.001

In [45]:
# criterion = nn.CrossEntropyLoss(ignore_index=w2i_ndfa['.pad'])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [46]:
# Use DataLoader for batching
dataset_ndfa2 = TensorDataset(x_train_ndfa_padded2)
dataloader_ndfa2 = DataLoader(dataset_ndfa2, batch_size=10, shuffle=True)

https://www.polarsparc.com/xhtml/DL-LSTM-NextWord.html - laptop gaat steeds out of memory na runnen van onderstaand dat vind ie dus niet zo leuk

In [19]:
# num_epochs = 10
# for epoch in range(1, num_epochs):
#     model.train()
#     optimizer.zero_grad()
#     y_predict, _ = model(x_train, h)
#     loss = criterion(y_predict, y_train)
#     print(f'Next Word Model LSTM -> Epoch: {epoch}, Loss: {loss}')
#     loss.backward()
#     optimizer.step()

In [ ]:
import torch.distributions as dist
def sample(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome logits
    :param temperature: Sampling temperature. 1.0 follows the given
        distribution, 0.0 returns the maximum probability element. :return: The index of the sampled element.
    """
    if temperature == 0.0:
        return lnprobs.argmax()
    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)
    
    return cd.sample()

In [47]:
max_length = 50
for epoch in range(num_epochs):
    total_loss = 0.0
    

    for batch_idx, (inputs,) in enumerate(dataloader_ndfa2):
        print(f'Batch Index: {batch_idx}, Batch Size: {inputs.size(0)}')
        model.train()        
        optimizer.zero_grad()

        # Autoregressive training
        h = None
        for t in range(inputs.size(1) - 1):  # -1 to exclude the last time step
            input_seq = inputs[:, :t+1] # Input sequence up to time step t
            target_word = inputs[:, t+1]  # Target is the next word (shifted by one time step)

            # Forward pass
            output, _ = model(input_seq, h)

            # Compute the loss
            loss = criterion(output, target_word)  

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Print the current iteration (outside the inner loop)
        print(f'Epoch [{epoch+1}/{num_epochs}], Iteration [{batch_idx+1}/{len(dataloader_ndfa2)}], Loss: {loss.item():.4f}')

        # Generate sample from the seed after each epoch - Q7
        model.eval()
        seed_seq = [w2i_ndfa['.start'], w2i_ndfa['('], w2i_ndfa['('], w2i_ndfa[')']]
        seed_input = torch.tensor([seed_seq], dtype=torch.long)
        with torch.no_grad():
            for t in range(max_length - 1):
                output, _ = model(seed_input, h)
                next_token = sample(output[:, -1, :])
                seed_seq.append(next_token.item())

                if next_token == w2i_ndfa['.end']:
                    break

                seed_input = torch.tensor([[next_token]], dtype=torch.long)
            
        generated_sequence = [''.join(i2w_ndfa[i] for i in seed_seq)]
        print(f'Generated Sequence after epoch {epoch+1}: {generated_sequence}')

    average_loss = total_loss / len(dataloader_ndfa2.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

torch.save(model.state_dict(), 'lstm_model.pth')


Batch Index: 0, Batch Size: 10


Epoch [1/1], Iteration [1/15000], Loss: 0.0000
Batch Index: 1, Batch Size: 10
Epoch [1/1], Iteration [2/15000], Loss: 0.0000
Batch Index: 2, Batch Size: 10
Epoch [1/1], Iteration [3/15000], Loss: 0.0000
Batch Index: 3, Batch Size: 10
Epoch [1/1], Iteration [4/15000], Loss: 0.0000
Batch Index: 4, Batch Size: 10
Epoch [1/1], Iteration [5/15000], Loss: 0.0000
Batch Index: 5, Batch Size: 10


KeyboardInterrupt: 

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0.0


    for batch_idx, (inputs,) in enumerate(dataloader_ndfa):
        print(f'Batch Index: {batch_idx}, Batch Size: {inputs.size(0)}')        
        optimizer.zero_grad()

        # Autoregressive training
        h = None

        input_seq = inputs[:, batch_idx:batch_idx+seq_len] # Input sequence up to time step t
        target_word = inputs[:, batch_idx+1:batch_idx+seq_len+1]  # Target is the next word (shifted by one time step)

        # Forward pass
        output, _ = model(input_seq, h)

        # Compute the loss
        loss = criterion(output, target_word)  

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print the current iteration (outside the inner loop)
        print(f'Epoch [{epoch+1}/{num_epochs}], Iteration [{batch_idx+1}/{len(dataloader_ndfa)}], Loss: {loss.item():.4f}')

    average_loss = total_loss / len(dataloader_ndfa.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

torch.save(model.state_dict(), 'lstm_model.pth')


In [ ]:
import torch.distributions as dist
def sample(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome logits
    :param temperature: Sampling temperature. 1.0 follows the given
        distribution, 0.0 returns the maximum probability element. :return: The index of the sampled element.
    """
    if temperature == 0.0:
        return lnprobs.argmax()
    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)
    
    return cd.sample()

In [42]:
seq = [w2i_brackets['.start'], w2i_brackets['('], w2i_brackets['('], w2i_brackets[')']]
seed_sequence_tensor = torch.tensor(seq).unsqueeze(0)

In [43]:
model.eval()
max_length = 50
end_token = w2i_brackets['.end']

In [44]:
with torch.no_grad():
    for _ in range(max_length):
        logits, _ = model(seed_sequence_tensor, None)
        last_word_logits = output[:, -1, :]
        p = F.softmax(last_word_logits, dim=-1)

        next_word = sample(p)
        if next_word == end_token:
            break
        
        seq.append(next_word.item())
        seed_sequence_tensor = torch.tensor(seq).unsqueeze(0)


In [45]:
generated_sequence = [i2w_brackets[i] for i in seq]

IndexError: list index out of range

In [15]:
print(model)

LSTM(
  (embedding): Embedding(15, 300)
  (lstm): LSTM(300, 300, batch_first=True)
  (fc): Linear(in_features=300, out_features=15, bias=True)
)
